<div style="background-color:#057BA3; padding:6px; color:#ffffff;">

## IUCAA Finesse Workshop 18-22nd Dec 2019
</div>


# Modelling a Fabry-Perot Cavity

In this notebook we'll be covering some basics of how to work more practically with <a href=http://www.gwoptics.org/finesse/>Finesse</a> and <a href =http://www.gwoptics.org/pykat/> PyKat</a>, by learning how to model a Fabry-Perot Cavity. Throughout the notebook are some explanations for both how to use Finesse/Pykat, and links to explanations of the relevant physics, if it's new to you.

This session is based on the GWoptics.org <a href=http://www.gwoptics.org/learn/index.html>Learn Laser Interferometry</a> course, with some extensions and adaptations. 

**Some key references**:
 - <a href=https://link.springer.com/article/10.1007/s41114-016-0002-8> Interferometer Techniques for Gravitational-Wave Detection</a>, C. Bond, D. Brown, A. Freise, and K. Strain, *Living Reviews in Relativity* 2016 
-- An extensive review (essentially a textbook) of laser interferometry techniques in the frequency domain used for detecting gravitational waves. It's written by developers and users of Finesse, so naturally covers the kinds of physics Finesse is able to model. It was updated in 2016 to include several new chapters, and is now available as a handy searchable html website (and you can copy-paste the latex equations).
 
 - <a href=http://www.gwoptics.org/finesse/reference/>Finesse Synatax reference</a>
-- Your best friend for finding the right way to code a beamsplitter, photodiode, etc in Finesse's own syntax (NB: not pykat).

 - <a href=http://www.gwoptics.org/finesse/reference/cheatsheet.php>Finesse Cheatsheet/FAQ</a>
-- The first place to look if you are confused about Finesse conventions or have other syntax or installation issues.
 
 - <a href=http://www.gwoptics.org/finesse/download/manual.pdf>Finesse 2 manual</a>, A. Freise, D. Brown, and C. Bond. 
--Manual for using Finesse 2, including both coded examples and some physics explanation along the way. 

## Goal of the session
Today you will model a 2-mirror optical cavity using a <a href = https://en.wikipedia.org/wiki/Plane_wave>plane-waves</a> model.

**Finesse/Pykat things you will learn**:
 - How to approach building a model
 - constructing an optical nodal network using Finesse syntax (the `kat` object)
 - using pykat to create, interact with and modify an existing `kat` object
 - some Finesse 'quirks', e.g. Finesse's definition of length

## Getting Started

We start by loading PyKat and the other Python packages we will need:

(type shift+enter to run)

In [ ]:
from pykat import finesse        # import the whole pykat.finesse package
from pykat.commands import *     # import all packages in pykat.commands
import numpy as np               # for basic math/sci/array functions
import matplotlib.pyplot as plt  # for plotting
import scipy                     # for analysing the plots

# tell the notebook to automatically show plots inline below each cell
%matplotlib inline               
# use pykat's plotting style. change dpi to change plot sizes on your screen
pykat.init_pykat_plotting(dpi=90)

## Planning the Model

The details that need including in your model will somewhat depend on the *purpose* of that model. E.g. an investigation exploring the effect of a lens on the spot size 3m further along the optical path does not necessarily need to include every flat mirror it reflects from along the way, but should include any other (de)focusing optics in that path. Therefore it is worth taking a few moments to plan the model before starting to code it.

 - what is the question your model is trying to answer? (and is Finesse the right tool to answer it?)
 - what plot/number will tell you that answer?
 - what parts of your experiment do you need to include for the result to be meaningful?

**Today's model**

Today, we will imagine we are designing a 2-mirror optical cavity which could be used, for example, as a mode-cleaner for a tabletop experiment.

You can learn more about 2-mirror cavities, also known as Fabry-Perot  cavities or interferometers, by reading <a href=https://link.springer.com/article/10.1007/s41114-016-0002-8#Sec33>section 5.1</a> of Interferometer Techniques for Gravitational-Wave Detection.

We will:
1. Construct a plane waves model of the cavity, plotting the power transmitted, reflected, and circulating from the cavity as its length is changed. 
2. Make the model more realistic using Gaussian beams and curved optics, then re-produce the powers plot to check that the behaviour still makes sense.
3. Plot the profile of the beam transmitted from the cavity.

## The Optical Layout

The setup we want to model is shown in the figure below. A *cavity*, or *optical resonator* is formed by the two partially-transparent mirrors M1 and M2. To measure the reflected, transmitted, and circulating power in the cavity, we add 3 photodiodes (these do not affect the beam propagation).

<img src=http://www.gwoptics.org/learn/02_Plane_waves/01_Fabry_Perot_cavity/fabry-perot.svg width=50%>

**The nodal network**

Finesse builds the optical model as a series of nodes, with component-specific instructions governing how light is propagated from one node to the next. It is recommended that you make a quick sketch of your setup, including these nodes, before you start coding your model - this is especially true for larger models. In our case we have:

```
                       M1                     M2
laser (n0) ------> (n1)|(n2) <----------> (n3)|(n4)
```

and will add photodiodes looking at the beams at nodes `n1` (refl), `n3` (circ) and `n4` (trans).


## Plane-Waves Finesse Model

<div style="background-color:#cceffc;padding:10px;">

### Task 1: write the base code for the optical model, and parse it to pykat
Fill in the blank '`basecode`' object below with the Finesse syntax commands for the setup sketched above, using the following components/parameters:
 * 1 W laser with 0 offset to the default wavelength (This is 1064nm in Finesse, unless you tell it otherwise)
 * a short space from laser to M1
 * mirrors 'M1' and 'M2' with T=0.15 and zero loss
 * a cavity length of 3km.
 
You may find the <a href=http://www.gwoptics.org/finesse/reference/>Finesse syntax reference</a> helpful! 
</div>

In [ ]:
basecode = """

"""

#initialise Finesse with a new empty kat object
basekat = finesse.kat() 
#tell Finesse to talk less
basekat.verbose = False
#parse the Finesse code into PyKat
basekat.parse(basecode)

It is generally good practice to keep the core optical model in an object called `base` or `basekat`, then create independant copies of it that are used for each investigation. This ensures that `basekat` stays intact and doesn't unintentionally retain some test alteration like a change in reflectivity or length. It also ensures that `basekat` contains *just* the components you need for your model, without being slowed down by the inclusion of excess detectors etc. 

**A reminder on lengths and tunings in Finesse**:

As discussed previously, distances in optical models can typically be separated into macroscopic and microscopic scales, so in Finesse lengths are separated into a macroscopic length, `L`, which is attributed to `space` components,  and a microscopic *tuning*, `phi` that is attributed to optics:

<img src="http://www.gwoptics.org/finesse/reference/images/LengthsTunings.svg" width=40%>

See the <a href="http://www.gwoptics.org/finesse/reference/cheatsheet.php">cheatsheet</a> for further details of how lengths and tunings are defined.

<div style="background-color:#cceffc;padding:10px;">

### Task 2: add the three photodiodes & simulate the response of the cavity as the tuning of M2 is varied.
Fill in the Finesse code for the three detectors as noted in the sketches above, then add an `xaxis` command to vary the tuning of M2 over a range of one wavelength. 
</div>

In [ ]:
#create an independent copy of basekat
kat1 = basekat.deepcopy()
#write the code for the PDs and xaxis
PDcode = """

"""
#parse the code for the new PDs and xaxis to the copied kat object 
kat1.parse(PDcode)
#run the simulation, and store the result in out1
out1 = kat1.run()


<div style="background-color:#cceffc;padding:10px;">

Now plot the results by running the code below:
</div>

In [ ]:
out1.plot(xlabel='Position of mirror M2 [deg]',
          ylabel='Power [W]', 
          title = 'Power vs. microscopic cavity length change')

The behaviour of the cavity depends strongly on its tuning. It is resonant periodically every 180$^\circ$, i.e. a position change of $\lambda_0/2$ or a round-trip path length change of $\lambda_0$, as expected.

<div style="background-color:#cceffc;padding:10px;">

### Task 3: simulate the response of the cavity as the laser frequency is varied
1. Repeat the task above, this time scanning the frequency of the laser instead of the tuning of a mirror. Set the frequency range so that your plot shows a few resonances of the cavity.


2.  Extract the Free Spectral Range of this cavity from the modelled data. 

**Tip:** The Scipy package has a function for detecting multiple peaks called `scipy.signal.find_peaks_cwt`, which you might find useful
</div>


<div style="background-color:#cceffc;padding:10px;">

### Task 4: Cavity types
In the above model, $T_1=T_2$ so the cavity is called 'impedence matched'. Fabry-Perot cavities and cavity couplings are covered in <a href=https://link.springer.com/article/10.1007/s41114-016-0002-8#Sec33>Section 5.1</a> of Interferometer Techniques. 

Run three simulations to show how the reflected, transmitted, and circulating cavity powers vary with microscopic tuning for:
1. impedance matched cavity ($T_1 = T_2$)
2. overcoupled cavity ($R_1 < R_2$)
3. undercoupled cavity ($R_1 > R_2$)

**Hint:** You might find the `kat.m2.setRTL(R, T, L)` method for changing the mirror parameters useful.
   
What interesting feature does an impedance matched cavity have?
 
In gravitational wave detectors we typically want very high optical powers, Is an over and under coupled cavity better for getting a higher circulating power in the cavity?
</div>


<div style="background-color:#cceffc;padding:10px;">
    
### Task 5: Frequency response, FWHM, and finesse

This task is about used modelled data to extract some cavity parameters. You should experiment with ways to achieve this task.

Use an overcoupled Fabry-Perot cavity Finesse model with R=1 and T=0 for the end mirror:

1. Plot the power circulating in the cavity as a function of the input laser frequency. You should change the laser frequency by at least two Free Spectral Ranges of the cavity.


2. Using this data, compute the cavity's full-width-half-maximum and finesse. Compare these results to analytic solutions. There isn't one specific way to do this task, you should experiment and problem solve!
</div>

